In [1]:
from tools import *
import sys

In [2]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils

In [3]:
def test(ecofile,infile,outfile,ite=1):
    # ecofile, ecoli model file prefix
    # infile, the model file for gap-filling
    # outfile, the output model file
    eco = load_ecoli_model(ecofile+'.mat',ecofile+'.xml')
    model = pickle.load(open(infile,'rb'))
    
    print('Convert eco to universal model') 
    universal = convert_eco_to_metacyc_kegg(eco,model)
    return universal,model

In [4]:
def print_rxn_of_met(model,met_id):
    met = model.metabolites.get_by_id(met_id)
    for rxn in met.reactions:
        if 'biomass' in rxn.id.lower():continue
        if rxn.metabolites[met]<0: continue
        print(rxn.id,rxn.reaction)

universal,model = test(ecofile_prefix,model_file3,outfile1)

utils.test_met_production(universal,'CPD-15815_c')

print_rxn_of_met(model,'CPD-15815_c')

In [5]:
def do_block(model,rxn_id):
    rxn = model.reactions.get_by_id(rxn_id) # differnt number of protons in metacyc
    model.remove_reactions([rxn])

In [6]:
def some_manual_curations(model):
    try:
        # based on metacyc website
        rxn = model.reactions.get_by_id('NADH-DEHYDROG-A-RXN')
        rxn.lower_bound = 0
        print(rxn.reaction)
    except: pass
    
    # enable only secretion of cit
    rxn = model.reactions.get_by_id('Exchange_CIT')
    rxn.upper_bound = 0
    print(rxn.reaction)
    
    do_block(model,'GLUt2rpp') # differnt number of protons in metacyc
    do_block(model,'RXN-11039')# break the loop


In [7]:
def block_some_universal_rxns(model):
    do_block(model,'GLUt2rpp') # differnt number of protons in metacyc
    do_block(model,'PROt2rpp') # differnt number of protons in metacyc


In [8]:
def add_missing_tag(rxn):
    rxn_clean = rxn.copy()
    # add _missing tag for those genes
    new_gr = rxn_clean.gene_reaction_rule
    for gene in parse_gr(rxn_clean.gene_reaction_rule):
        new_gr = new_gr.replace(gene,gene+'_missing')
    rxn_clean.gene_reaction_rule = new_gr
    return rxn_clean

In [9]:
def manually_fill_ions_uptake(universal,model):
    rxns_to_add = []
    def _add_rxn(rxn_id):
        rxn = universal.reactions.get_by_id(rxn_id)
        rxn_clean = add_missing_tag(rxn)
        
        rxns_to_add.append(rxn_clean)
        print(rxn_clean.id, rxn_clean.reaction,rxn_clean.gene_reaction_rule)
        print()
    
    # K+ 
    _add_rxn('TRANS-RXN-3')
    
    # CO+2_c
    _add_rxn('TRANS-RXN-141A_copy2')
    
    # CU+2_c
    _add_rxn('RXN0-14_copy2')
    
    # FE+2_c 
    _add_rxn('TRANS-RXN-8_copy2')
    
    # FE+3_c 
    _add_rxn('3.6.3.30-RXN')
    
    # MN+2_c
    _add_rxn('RXN0-16_copy2')
    
    model.add_reactions(rxns_to_add)

In [10]:
def gap_filling_for_lipid(universal,model):
    for met_id in ['CPD-12819_p','CPD-17086_p','KDO2-LIPID-IVA_e']:
        s = utils.test_met_production(model,met_id)
        rxns_to_add = []
        if s.objective_value == 0:
            su = utils.test_met_production(universal,met_id)
            for rxn_id in su.fluxes.index:
                if rxn_id == 'tmp_sink': continue
                if rxn_id == 'ATPASE-RXN_copy1': continue
                if rxn_id.startswith('EX_'):continue
                if su.fluxes[rxn_id] == 0: continue
                try: model.reactions.get_by_id(rxn_id)
                except: 
                    rxn = universal.reactions.get_by_id(rxn_id)
                    rxn = add_missing_tag(rxn)
                    rxns_to_add.append(rxn)
                    print('  ',rxn.id,rxn.reaction)
        if len(rxns_to_add) >0: model.add_reactions(rxns_to_add)
        print(len(rxns_to_add), 'for lipid',met_id)
        model.repair()
       

In [11]:
def run_gapfilling(ecofile,infile,outfile,ite=1):
    # ecofile, ecoli model file prefix
    # infile, the model file for gap-filling
    # outfile, the output model file
    eco = load_ecoli_model(ecofile+'.mat',ecofile+'.xml')
    block_some_universal_rxns(eco)
    
    model = pickle.load(open(infile,'rb'))
    
    some_manual_curations(model)
    
    print('Convert eco to universal model') 
    universal = convert_eco_to_metacyc_kegg(eco,model)
    
    print()
    print('Test if there is consistant prediction before and after conversion')
    report_model_status(eco)
    report_model_status(universal)
    
    universal.objective = 'BIOMASS_Ec_iML1515_core_75p37M'
    eco.objective = 'BIOMASS_Ec_iML1515_core_75p37M'
    print(universal.optimize())
    print(eco.optimize())
    print()
    
    
    print('Test if model can produce components in biomass with the medium without YE')
    utils.set_yeast_extraction(model,lb=0,ub=0)
    check_if_model_can_produce_mets_in_biomass_one_by_one(model)
    print()
    
    print('Do gap-filling one by one')
    manually_fill_ions_uptake(universal,model)
    #gap_filling_for_lipid(universal,model)
    for i in range(ite):
        for met in  model.reactions.Biomass_v1.reactants:
            if met.id == 'ions_c': continue
            #if met.id == 'lipid_c': continue
            do_gap_filling(met,model,universal)
        
    print('Test if model can produce components in biomass with the medium without YE, 1 glucose')
    model.reactions.Exchange_Glucopyranose.lower_bound = 0
    model.reactions.Exchange_Glucopyranose.upper_bound = 1
    check_if_model_can_produce_mets_in_biomass_one_by_one(model)
    print()
    
    print('Test biomass production with/wihout YE')
    print(utils.test_biomass_production(model))
    print()
    
    print('Update missing genes')
    model = update_missing_genes(model,1e-3,report=True,coverage=45)
    print()
    
    print('Save model')
    save_pickle_model(model,outfile)

In [12]:
ecofile_prefix ='../../../ComplementaryData/iML1515'
model_file1 = '../Results/halo_metacycmodel_100_curated_compart_with_eco_medium_biomass.pkl'
outfile1    = '../Results/halo_metacycmodel_100_curated_compart_with_eco_medium_biomass_gapfilled.pkl'

model_file2 = '../Results/halo_metacyc_kegg_curated_compart_with_eco_medium_biomass.pkl'
outfile2    = '../Results/halo_metacyc_kegg_curated_compart_with_eco_medium_biomass_gapfilled.pkl'

model_file3 = '../Results/halo_metacycmodel_100_curated_compart_with_eco_without_missing_medium_biomass.pkl'
outfile3    = '../Results/halo_metacycmodel_100_curated_compart_with_eco_without_missing_medium_biomass_gapfilled.pkl'

model_file4 = '../Results/halo_metacyc_kegg_curated_compart_with_eco_without_missing_medium_biomass.pkl'
outfile4    = '../Results/halo_metacyc_kegg_curated_compart_with_eco_without_missing_medium_biomass_gapfilled.pkl'

In [13]:
run_gapfilling(ecofile_prefix,model_file1,outfile1)

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpyp1b3kua.lp
Reading time = 0.03 seconds
: 2399 rows, 4618 columns, 21886 nonzeros


cobra/core/group.py:109 UserWarning: need to pass in a list


NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
 <-- CIT_e
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2710
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2710
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1515
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x132bbf8470>
<Solution 0.877 at 0x132bbf8470>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 0.0
   rna_c rna 0.0
   ions_c ions 0.0
   lipids_c lipids 0.0
   10-FORMYL-THF_c 10-Formyltetrahydrofolate 0.0
   2fe2s_c [2Fe-2S] iron-sulfur cluster 0.0
 

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   KDO2-LIPID-IVA_e KDO(2)-lipid IV(A) 0.0
   CPD-12819_p Phosphatidylethanolamine (dihexadecanoyl, n-C16:0) 0.0
   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

protein_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmposl5n275.lp
Reading time = 0.02 seconds
: 2399 rows, 4626 columns, 21902 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpwfusib9

...

After gap-filling: 28.594115044247744
Added reactions:
   PANTOTHENATE-KIN-RXN ATP_c + PANTOTHENATE_c --> 4-P-PANTOTHENATE_c + ADP_c + PROTON_c



FAD_c
Before gap-filling: 24.666111766092904
No need for gap-filling



CPD-15815_c
Before gap-filling: 1000.0
No need for gap-filling



METHYLENE-THF_c
Before gap-filling: 31.85431567907706
No need for gap-filling



CPD0-2278_p
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp5zdsbk_e.lp
Reading time = 0.02 seconds
: 2400 rows, 4668 columns, 22058 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp6zpcssn0.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'TETHYDPICSUCC-RXN' since it already exists.
Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_CARBON-DIOXIDE_c' since it already exists.


After gap-filling: 15.875735129068463
Added reactions:
   TRANS-RXN-62A D-ALANINE_p + PROTON_p --> D-ALANINE_c + PROTON_c



NAD_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp7nb_gffa.lp
Reading time = 0.02 seconds
: 2400 rows, 4670 columns, 22066 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpkl1qhabc.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_CARBON-DIOXIDE_c' since it already exists.


After gap-filling: 34.088583403185226
Added reactions:
   NAD-SYNTH-NH3-RXN AMMONIA_c + ATP_c + DEAMIDO-NAD_c --> AMP_c + NAD_c + PPI_c + PROTON_c



NADP_c
Before gap-filling: 32.333675213675235
No need for gap-filling



PROTOHEME_c
Before gap-filling: 22.74570041090773
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 112.06515699333966
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 46.87832399731123
No need for gap-filling



SIROHEME_c
Before gap-filling: 20.788747012632292
No need for gap-filling



SUC-COA_c
Before gap-filling: 26.814398340248914
No need for gap-filling



THF_c
Before gap-filling: 33.25824254037778
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 39.95221020092736
No need for gap-filling



CPD-9649_c
Before gap-filling: 19.174930555555555
No need for gap-filling



ATP_c
Before gap-filling: 51.89763348996154
No need for gap-filling



Test if model can produce components in biomass with the medium without 

In [14]:
run_gapfilling(ecofile_prefix,model_file2,outfile2)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpy4ywjqgz.lp
Reading time = 0.02 seconds
: 3310 rows, 6314 columns, 29474 nonzeros


cobra/core/group.py:109 UserWarning: need to pass in a list


NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
 <-- CIT_e
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2710
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2710
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1515
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x1329cf9c88>
<Solution 0.877 at 0x1329cf9f60>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 965.5793048215228
   rna_c rna 169.77619447193663
   ions_c ions 0.0
   lipids_c lipids 0.0
   10-FORMYL-THF_c 10-Formyltetrahydrofolate 0.0
   2fe2s_c [2Fe-

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-12819_p Phosphatidylethanolamine (dihexadecanoyl, n-C16:0) 0.0
   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

protein_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpf39y7e2o.lp
Reading time = 0.03 seconds
: 3310 rows, 6322 columns, 29490 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp6uatvb02.lp
Reading time = 0.02 seconds
: 1872 row

...

After gap-filling: 32.78688524590166
Added reactions:
   PANTOTHENATE-KIN-RXN ATP_c + PANTOTHENATE_c --> 4-P-PANTOTHENATE_c + ADP_c + PROTON_c



FAD_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpkcocjod8.lp
Reading time = 0.03 seconds
: 3311 rows, 6346 columns, 29576 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpk1w25i8v.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 80.25346080305927
Added reactions:
   DM_CARBOXYL-GROUP_c CARBOXYL-GROUP_c --> 



CPD-15815_c
Before gap-filling: 1000.0
No need for gap-filling



METHYLENE-THF_c
Before gap-filling: 118.34368421052659
No need for gap-filling



CPD0-2278_p
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpybu1gnx3.lp
Reading time = 0.03 seconds
: 3311 rows, 6348 columns, 29578 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpbqrf_xl0.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 54.054054054054056
Added reactions:
   TRANS-RXN-62A D-ALANINE_p + PROTON_p --> D-ALANINE_c + PROTON_c



NAD_c
Before gap-filling: 109.87581151832444
No need for gap-filling



NADP_c
Before gap-filling: 103.12668304668284
No need for gap-filling



PROTOHEME_c
Before gap-filling: 62.5
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 333.3333333333333
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 156.41947826086934
No need for gap-filling



SIROHEME_c
Before gap-filling: 38.4615384615384
No need for gap-filling



SUC-COA_c
Before gap-filling: 90.90909090909099
No need for gap-filling



THF_c
Before gap-filling: 124.91833333333366
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 108.3502350813743
No need for gap-filling



CPD-9649_c
Before gap-filling: 35.714285714285715
No need for gap-filling



ATP_c
Before gap-filling: 162.26505154639193
No need for gap-filling



Test if model can produce components in

In [15]:
run_gapfilling(ecofile_prefix,model_file3,outfile3,ite=2)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp0_5dgv9k.lp
Reading time = 0.02 seconds
: 2337 rows, 4472 columns, 21114 nonzeros


cobra/core/group.py:109 UserWarning: need to pass in a list


NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
 <-- CIT_e
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2710
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2710
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1515
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x1325a4f748>
<Solution 0.877 at 0x1325a4f748>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 0.0
   rna_c rna 0.0
   ions_c ions 0.0
   lipids_c lipids 0.0
   10-FORMYL-THF_c 10-Formyltetrahydrofolate 0.0
   2fe2s_c [2Fe-2S] iron-sulfur cluster 0.0
 

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-12819_p Phosphatidylethanolamine (dihexadecanoyl, n-C16:0) 0.0
   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

protein_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp72_jd_h3.lp
Reading time = 0.02 seconds
: 2337 rows, 4480 columns, 21130 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpt50hl98j.lp
Reading time = 0.02 seconds
: 1872 row

...

After gap-filling: 20.64270779976717
Added reactions:
   DM_D-ALANINE_p D-ALANINE_p --> 



NAD_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp1b7e4chc.lp
Reading time = 0.02 seconds
: 2338 rows, 4530 columns, 21324 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmphv3oai0s.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_CARBON-DIOXIDE_c' since it already exists.
Ignoring reaction 'TRANS-RXN-114' since it already exists.


After gap-filling: 35.490411861614234
Added reactions:
   NAD-SYNTH-NH3-RXN AMMONIA_c + ATP_c + DEAMIDO-NAD_c --> AMP_c + NAD_c + PPI_c + PROTON_c



NADP_c
Before gap-filling: 33.72630919765197
No need for gap-filling



PROTOHEME_c
Before gap-filling: 26.539286788399572
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 136.02540184453224
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 50.43369014084507
No need for gap-filling



SIROHEME_c
Before gap-filling: 23.43043771043771
No need for gap-filling



SUC-COA_c
Before gap-filling: 27.81503686274513
No need for gap-filling



THF_c
Before gap-filling: 34.79419999999999
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpsa40n_l0.lp
Reading time = 0.02 seconds
: 2338 rows, 4532 columns, 21338 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp526n_xlx.lp
Rea

Ignoring reaction 'SULFITE-REDUCT-RXN' since it already exists.
Ignoring reaction 'ADENYLYLSULFKIN-RXN' since it already exists.
Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'TRANS-RXN-114' since it already exists.
Ignoring reaction 'DM_CARBOXYL-GROUP_c' since it already exists.


After gap-filling: 40.23848704179314
Added reactions:
   THZPSN3 ATP_c + CPD-12279_c + DEOXYXYLULOSE-5P_c + NADPH_c + PROTON_c + iscssh_c --> AMP_c + CARBON-DIOXIDE_c + 2.0 CPD-15815_c + NADP_c + PPI_c + THZ-P_c + iscs_c
   DM_OROTATE_c OROTATE_c --> 



CPD-9649_c
Before gap-filling: 17.623700243502473
No need for gap-filling



ATP_c
Before gap-filling: 55.8462216461439
No need for gap-filling



protein_c
Before gap-filling: 57.929527845803015
No need for gap-filling



dna_c
Before gap-filling: 574.7771849035436
No need for gap-filling



rna_c
Before gap-filling: 94.1436718800054
No need for gap-filling



lipids_c
Before gap-filling: 215.3222549186161
No need for gap-filling



10-FORMYL-THF_c
Before gap-filling: 33.29588516746416
No need for gap-filling



2fe2s_c
Before gap-filling: 75.63956521739131
No need for gap-filling



2-OCTAPRENYL-6-HYDROXYPHENOL_c
Before gap-filling: 20.78060002343841
No need for gap-filling



CPD-7_c
Before gap-filling: 38.23538461538462
No need for

In [16]:
run_gapfilling(ecofile_prefix,model_file4,outfile4,ite=2)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmprfhs026o.lp
Reading time = 0.02 seconds
: 3252 rows, 6170 columns, 28714 nonzeros


cobra/core/group.py:109 UserWarning: need to pass in a list


NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
 <-- CIT_e
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2710
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2710
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1515
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x132d154a90>
<Solution 0.877 at 0x132d154d68>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 0.0
   rna_c rna 0.0
   ions_c ions 0.0
   lipids_c lipids 0.0
   10-FORMYL-THF_c 10-Formyltetrahydrofolate 0.0
   2fe2s_c [2Fe-2S] iron-sulfur cluster 0.0
 

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

protein_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp7sx47ufg.lp
Reading time = 0.02 seconds
: 3252 rows, 6178 columns, 28730 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp4b134ox6.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
Gap-filling failed for protein_c



dn

...

After gap-filling: 27.027027027027028
Added reactions:
   TRANS-RXN-62A D-ALANINE_p + PROTON_p --> D-ALANINE_c + PROTON_c



NAD_c
Before gap-filling: 66.66666666666656
No need for gap-filling



NADP_c
Before gap-filling: 62.5
No need for gap-filling



PROTOHEME_c
Before gap-filling: 31.249999999999996
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 166.66666666666666
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 78.16235294117654
No need for gap-filling



SIROHEME_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmplspqldix.lp
Reading time = 0.03 seconds
: 3253 rows, 6194 columns, 28798 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpondq5y5w.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 35.71428571428567
Added reactions:
   DM_DIHYDROXYPENTANEDIONE_c DIHYDROXYPENTANEDIONE_c --> 



SUC-COA_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpw2udv6fw.lp
Reading time = 0.02 seconds
: 3253 rows, 6196 columns, 28800 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpdmkstk45.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
Gap-filling failed for SUC-COA_c



THF_c
Before gap-filling: 75.68886075949354
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpk7w8t6en.lp
Reading time = 0.03 seconds
: 3253 rows, 6196 columns, 28800 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp80ws36ii.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
Gap-filling failed for THIAMINE-PY

Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'TRANS-RXN-114' since it already exists.
Ignoring reaction 'DM_murein5p5p5p_p' since it already exists.


Gap-filling failed for lipids_c



10-FORMYL-THF_c
Before gap-filling: 75.68886075949354
No need for gap-filling



2fe2s_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpghtmyi_t.lp
Reading time = 0.03 seconds
: 3253 rows, 6196 columns, 28800 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpvpi9w2jl.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
Gap-filling failed for 2fe2s_c



2-OCTAPRENYL-6-HYDROXYPHENOL_c
Before gap-filling: 21.739130434782606
No need for gap-filling



CPD-7_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpr5fsob_2.lp
Reading time = 0.03 seconds
: 3253 rows, 6196 columns, 28800 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp_s88ulr9.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
Gap-filling failed for CPD-7_